In [ ]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np

In [ ]:

class MCTS:
    def __init__(self, depth=20, B=10, tau=None, noise_std=1, c=1.0, rollouts_per_leaf=1):
        self.depth = depth
        self.B = B
        self.tau = tau if tau else depth / 5
        self.noise_std = noise_std
        self.c = c
        self.rollouts_per_leaf = rollouts_per_leaf
        self.target_address = self.generate_target()

    # random target address
    def generate_target(self):
        return "".join(random.choice("LR") for _ in range(self.depth))

    # measure the value of a node based on the distance to the target
    def calculate_value(self, address):
        distance = sum(1 for a, b in zip(address, self.target_address) if a != b)
        noise = random.gauss(0, self.noise_std)
        return self.B * math.exp(-distance / self.tau) + noise

    #subclass node to represent a single node in the tree
    class Node:
        def __init__(self, address="", parent=None):
            self.address = address
            self.parent = parent
            self.visits = 0
            self.value = 0
            self.children = {}

        # node UCB score
        def ucb_score(self, c):
            if self.visits == 0:
                return float('inf')  # Prioritize unvisited nodes
            exploitation = self.value / self.visits
            exploration = c * math.sqrt(math.log(self.parent.visits) / self.visits)
            return exploitation + exploration

    # select best node
    def select(self, node):
        while node.children:
            node = max(node.children.values(), key=lambda child: child.ucb_score(self.c))
        return node

    # add children to node
    def expand(self, node):
        if len(node.address) < self.depth:
            for direction in "LR":
                child_address = node.address + direction
                node.children[child_address] = self.Node(address=child_address, parent=node)

    simulate rollouts from node and average values
    def simulate(self, node):
        total_value = 0
        for _ in range(self.rollouts_per_leaf):
            simulated_path = node.address + "".join(
                random.choice("LR") for _ in range(self.depth - len(node.address))
            )
            total_value += self.calculate_value(simulated_path)
        return total_value / self.rollouts_per_leaf

    #backup
    def backup(self, node, value):
        while node:
            node.visits += 1
            node.value += value
            node = node.parent

    #MCTS search
    def search(self, iterations=50):
        root = self.Node()
        depth_list = []
        for _ in range(iterations):
            # Selection
            selected_node = self.select(root)
            # Track depth of the selected node
            depth_list.append(len(selected_node.address))
            # Expansion
            self.expand(selected_node)
            # Simulation
            if selected_node.children:
                selected_node = random.choice(list(selected_node.children.values()))
            value = self.simulate(selected_node)
            # Backup
            self.backup(selected_node, value)
        
        # Return the best child and depth statistics
        return max(root.children.values(), key=lambda n: n.value / n.visits if n.visits > 0 else 0), root, depth_list

In [ ]:

# MCTS rollouts with 5 per leaf
mcts = MCTS(depth=20, c=2.0, rollouts_per_leaf=5)
# Perform the search with 50 iterations
best_node, root, _ = mcts.search(iterations=50)  # Include the depth_list, but ignore it with _
best_node_depth = len(best_node.address)

#print(f"Target address: {mcts.target_address}")
#print(f"Best address found: {best_node.address}")
#print(f"Depth of best node: {best_node_depth}")
#print(f"Average value of best node: {best_node.value / best_node.visits if best_node.visits > 0 else 0}")
#print(f"Total visits to best node: {best_node.visits}")

In [ ]:
depth = 20
num_runs = 100
iterations_per_run = 50
rollouts_per_leaf = 5
target_address = "".join(random.choice("LR") for _ in range(depth))

# c values with step size 0.2
c_values = np.arange(0.1, 5.1, 0.1)  # Continuous values with step size 0.2
results = []

for c in c_values:
    optimal_values = []
    depths = []
    times = []
    for run in range(num_runs):
        # MCTS instance with fixed target and c
        mcts = MCTS(depth=depth, c=c, rollouts_per_leaf=rollouts_per_leaf)
        mcts.target_address = target_address

        # time
        start_time = time.time()
        best_node, root, depth_list = mcts.search(iterations=iterations_per_run)
        elapsed_time = time.time() - start_time

        best_value = best_node.value / best_node.visits if best_node.visits > 0 else 0
        optimal_values.append(best_value)
        times.append(elapsed_time)
        depths.extend(depth_list)

    avg_value = sum(optimal_values) / len(optimal_values)
    avg_time = sum(times) / len(times)
    avg_depth = sum(depths) / len(depths)
    std_time = (sum((t - avg_time) ** 2 for t in times) / len(times)) ** 0.5

    results.append((c, avg_value, avg_time, avg_depth, std_time))
    print(f"c = {c}: Average Value = {avg_value:.4f}, Avg Time = {avg_time:.4f}s, Avg Depth = {avg_depth:.2f}, Std Time = {std_time:.4f}s")

c_vals, avg_vals, avg_times, avg_depths, std_times = zip(*results)


plt.figure(figsize=(15, 5))

# average
plt.subplot(1, 3, 1)
plt.plot(c_vals, avg_vals, label='Average Optimal Value', color="green", linewidth=2)
plt.xlabel('c (Exploration Parameter)', fontsize=12)
plt.ylabel('Average Optimal Value', fontsize=12)
plt.title('Impact of c on Value (Continuous)', fontsize=14)
plt.grid()
plt.legend()

# Time
plt.subplot(1, 3, 2)
plt.errorbar(c_vals, avg_times, yerr=std_times, fmt='-', capsize=5, label='Computation Time', color="purple", linewidth=2)
plt.xlabel('c (Exploration Parameter)', fontsize=12)
plt.ylabel('Time (seconds)', fontsize=12)
plt.title('Impact of c on Computation Time (Continuous)', fontsize=14)
plt.grid()
plt.legend()

# Depth
plt.subplot(1, 3, 3)
plt.plot(c_vals, avg_depths, label='Average Depth of Best Nodes', color="blue", linewidth=2)
plt.xlabel('c (Exploration Parameter)', fontsize=12)
plt.ylabel('Average Depth', fontsize=12)
plt.title('Impact of c on Depth (Continuous)', fontsize=14)
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()